## Fragmentación de escritura del índice secundario global
La clave principal de una tabla de DynamoDB o un índice secundario global consta de una clave de partición y una clave de ordenación opcional. La forma de diseñar el contenido de estas claves es extremadamente importante para la estructura y el rendimiento de su base de datos. Los valores de la clave de partición determinan las particiones lógicas en las que se almacenan los datos. Por lo tanto, es importante elegir un valor de clave de partición que distribuya uniformemente la carga de trabajo entre todas las particiones de la tabla o índice secundario global. Para obtener más información sobre cómo elegir la clave de partición correcta, consulte nuestro blog titulado Elegir la clave de partición correcta de DynamoDB .

En este ejercicio, aprenderá sobre la fragmentación de escritura del índice secundario global, que es un patrón de diseño eficaz para consultar de forma selectiva los elementos distribuidos en diferentes particiones lógicas de una tabla muy grande. Revisemos el ejemplo de los registros de acceso al servidor [del Ejercicio 1]{href=«/design-patterns/ex1capacity»}, que está basado en los registros de acceso al servicio de Apache. Esta vez, consulta los elementos con código de respuesta 4xx. Tenga en cuenta que los elementos con código de respuesta 4xx son un porcentaje muy pequeño de los datos totales y no tienen una distribución uniforme por código de respuesta. Por ejemplo, el código de respuesta 200 OK tiene más registros que los demás, lo cual es esperable para cualquier aplicación web.

El siguiente gráfico muestra la distribución de los registros de log por código de respuesta para el archivo de muestra, logfile_medium1.csv.

![img](images/image8.jpg)

In [1]:
import boto3
from botocore.exceptions import ClientError
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
from spdynamodb import DynamoTable
from datetime import datetime
import json
import time
from decimal import Decimal

In [2]:
dt = DynamoTable()
try:
    dt.select_table('logfile')
    print(dt)
except:
    dt.create_table(
        table_name='logfile',
        partition_key='PK',
        partition_key_type='S',
        read_capacity=5,
        write_capacity=5,
        tags=[{'Key': 'workshop-design-patterns', 'Value': 'targeted-for-cleanup'}]
)

Table info:
 - Table name: logfile
 - Table arn: arn:aws:dynamodb:us-east-1:637423169504:table/logfile
 - Table creation: 2024-07-23T13:09:50
 - Key schema: [{'AttributeName': 'PK', 'KeyType': 'HASH'}]
 - Attribute definitions: [{'AttributeName': 'GSI1-PK', 'AttributeType': 'S'}, {'AttributeName': 'PK', 'AttributeType': 'S'}]
 - Point-in-time recovery status: DISABLED
 - Delete protection: False
 - Stream enabled: OFF
 - Tags: [{'Key': 'workshop-design-patterns', 'Value': 'targeted-for-cleanup'}]



Creará un índice secundario global de escritura separada en una tabla para aleatorizar las escrituras a través de múltiples valores de clave de partición lógica. En efecto, esto aumenta el rendimiento de escritura y lectura de la aplicación. Para aplicar este patrón de diseño, puedes crear un número aleatorio de un conjunto fijo (por ejemplo, de 1 a 10), y utilizar este número como la clave de partición lógica para un índice secundario global. Al aleatorizar la clave de partición, las escrituras en la tabla se distribuyen uniformemente entre todos los valores de la clave de partición, independientemente de cualquier atributo. Esto producirá un mejor paralelismo y un mayor rendimiento general. Además, si la aplicación necesita consultar los registros por un código de respuesta específico en una fecha específica, puede crear una clave de ordenación compuesta utilizando una combinación del código de respuesta y la fecha.

En este ejercicio, se crea un índice secundario global utilizando valores aleatorios para la clave de partición y la clave compuesta responsecode#date#hourofday como clave de ordenación. La tabla logfile_scan que creó y rellenó durante la fase de preparación del taller ya tiene estos dos atributos. Si no completó los pasos de configuración, vuelva a Configuración - Paso 6 y complete el paso. Estos atributos se crearon utilizando el siguiente código.

In [ ]:
SHARDS = 10
newitem['GSI_1_PK'] = "shard#{}".format((newitem['requestid'] % SHARDS) + 1)
newitem['GSI_1_SK'] = row[7] + "#" + row[2] + "#" + row[3]